## Colab 환경에서 하는 것을 권장합니다.

# 사전 준비

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd drive/MyDrive/TextMining_WordEmbedding

/content/drive/MyDrive/TextMining_WordEmbedding


In [ ]:
#konlpy 설치 (mecab 제외). 3~40초 정도 소요
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

In [7]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [8]:
# 필요 라이브러리 불러오기
import os, json
from tqdm.notebook import tqdm

from konlpy.tag import Okt

from gensim.models import Word2Vec

import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [9]:
# data 불러오기
data_dir = os.path.join(os.getcwd(), 'data')

corpus = []
for i in tqdm(os.listdir(data_dir)):
    with open(os.path.join(data_dir, i), 'r') as f:
        json_data = json.load(f)
    corpus.append(json_data['sentences'])
corpus = sum(corpus, [])

  0%|          | 0/50 [00:00<?, ?it/s]

데이터를 확인하고 어떻게 전처리를 하고 토큰화를 해야 할 지 고민해보세요!

In [10]:
corpus[:50]

['슈퍼컴퓨터 전문가인 김문주 박사가 한국의 슈퍼컴퓨터 정책에 쓴소리를 했다',
 '그는 IBM에서 30년간 몸담고 일하며 슈퍼컴퓨터 개발에 핵심 역할을 했다',
 '슈퍼컴퓨터와 관련한 수백건 발명과 특허 등을 개발한 인물이다',
 '2009년 IBM 퇴임 당시에는 `수석발명가`라는 호칭을 받았다',
 '수석발명가는 IBM에서 최고 개발자에게 붙여준다',
 '김 박사가 두 번째로 얻었다',
 '김 박사는 “한국에 알파고 이후 인공지능(AI) 열풍이 불고 있는데, 이미 미국에서는 AI가 체스를 이겨 충격을 받은 지 오래”라며 “AI는 슈퍼컴퓨터 없이는 구현할 수가 없는데, 한국이 개발하겠다는 AI 소프트웨어만 갖고는 할 수 있는 게 없다”고 지적했다',
 '구글이 개발에 몰두하고 있는 것은 `슈퍼컴퓨터`란 뜻이다',
 '실제로 구글은 지난해 AI로 쓰기 위해 여러 CPU를 하나의 칩에다 패키징한 `텐서프로세싱유닛(TPU)` 칩세트를 공개했다',
 '구글은 CPU, GPU, FPGA를 하나의 칩에다 넣고 멀티 패키징을 했다',
 '구글은 `스트리트뷰`라는 슈퍼컴퓨터도 자체적으로 만들었다',
 '김 박사는 “AI를 구현하려면 하드웨어는 당연히 같이 개발돼야 한다”며 “한국 프로젝트는 한국적인 슈퍼컴퓨터 플랫폼을 만들어 핵심(Core) 기술을 개발하려는 것이 아니라 외국 컴퓨터를 몇 대 사오려고 하는 것 같다”고 말했다',
 '그는 중국과 IBM의 슈퍼컴퓨터 개발 프로젝트(2006~2008년, 863프로그램)에 핵심 멤버로 참여했다',
 '그러다 미국 정부가 핵심 기술을 유출해선 안 된다고 막았고, IBM은 프로젝트를 중단했다',
 '중국은 단 10년 만에 93PF(페타플롭)의 선웨이 타이후라이트(Sunway TaihuLight)를 개발해 IBM을 뛰어넘었다',
 '중국은 멀티코어 CPU도 자체 개발해서 인텔 기술을 뛰어 넘으려 하고 있다',
 '슈퍼컴퓨터는 실생활에 혁신을 가져올 수 있다',
 '일례로 `의료 혁신`이다',
 '그는 “현재 위, 담낭 등의 사이즈를 

# 1번

자신만의 CustomTokenization을 만들어보세요! 세션에서 말씀 드렸듯이 정답이 있는 것이 아닙니다. 

제가 세션에서 설명했던 것 처럼 pos를 이용하여 품사를 구하고 품사를 통해 stop words를 지정해서 제거해 줄 수 있고, 따로 stopw words를 지정해서 의미 없다고 생각하는 단어들을 제거해도 상관 없습니다!

물론 Okt말고 다른 Tokenizer를 사용해도 괜찮습니다. 

다양한 방법으로 전처리를 진행해보고 왜 그런 전처리와 tokenizing 방법을 사용했는지 설명해주세요!

[여러분의 설명]   
저는 특정 키워드(명사)가 어떤 단어들(명사)과 가장 큰 관련이 있는지를 알아보고 싶었습니다.  
그래서 okt.nouns를 사용하여 명사만 tokenizing을 하였습니다.   

특별히 한 글자로 구성된 단어는 대부분 큰 의미가 없다는 점에 근거하여 (일, 건 etc) 한 글자 단어를 모두 제거했습니다. 진행과정은 다음과 같습니다:  
1) 일단 corpus_tokenized_allwords에 모든 단어를 담아준 후,  
2) corpus_tokenized에 한글자 이상 단어만 옮겨주었습니다.

In [11]:
okt = Okt()

In [12]:
# 자신만의 tokenization, preprocess 규칙을 만들고 그 규칙에 맞는 CustomTokenizer를 만들어보세요.
# 한 문장을 input으로 받아 token이 들어있는 list를 반환하는 함수를 구현해보세요.
def CustomTokenization(sentence):
  return okt.nouns(sentence)

In [13]:
corpus_tokenized_allwords = []
for i in tqdm(corpus):
  corpus_tokenized_allwords.append(CustomTokenization(i))

  0%|          | 0/1020 [00:00<?, ?it/s]

In [14]:
corpus_tokenized_allwords[:3]

[['슈퍼컴퓨터', '전문가', '김문주', '박사', '한국', '슈퍼컴퓨터', '정책', '쓴소리'],
 ['그', '몸', '일', '슈퍼컴퓨터', '개발', '핵심', '역할'],
 ['슈퍼컴퓨터', '관련', '건', '발명', '특허', '등', '개발', '인물']]

In [15]:
corpus_tokenized = []
for i in corpus_tokenized_allwords:
  imsi = []
  for j in i:
    if len(j) > 1:
      imsi.append(j)
  corpus_tokenized.append(imsi)

In [16]:
corpus_tokenized[:3]

[['슈퍼컴퓨터', '전문가', '김문주', '박사', '한국', '슈퍼컴퓨터', '정책', '쓴소리'],
 ['슈퍼컴퓨터', '개발', '핵심', '역할'],
 ['슈퍼컴퓨터', '관련', '발명', '특허', '개발', '인물']]

# 2번

이제 word2vec 모델을 학습시켜 봅시다!

1. 사전학습 없이 모델 학습
2. 사전학습 된 모델을 사용하여 학습

두가지를 진행해보세요

hyperparameter는 본인이 하고 싶으신 것으로 하면 됩니다. 단, 사전학습 된 모델을 사용할 때는 size = 200으로 맞추어 주세요!

hyperparameter 설정 기준을 설명해주세요!

[여러분의 설명]   
size = 200: embedding vector 차원 200(사전학습 된 모델과 동일하게 설정)   
min_count = 0: 한 번도 안 나온 단어를 무시, 기본값으로 설정하면 V가 너무 작게 나왔습니다.  
나머지 하이퍼파라미터는 디폴트 값을 사용하였습니다.

In [29]:
# 사전학습 없이 모델 학습
# 모델 생성
model_no_pretrained = Word2Vec(size = 200, min_count = 0)

# 모델 vocabulary 추가
model_no_pretrained.build_vocab(corpus_tokenized)

total_examples = model_no_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_no_pretrained.wv.vocab))

현재 vocabulary size(V) : 2725


In [30]:
# 모델 학습
model_no_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 5)

(45951, 48030)

In [31]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_no_pretrained.wv.most_similar('컴퓨터')

[('개발', 0.38873451948165894),
 ('경우', 0.3530920743942261),
 ('상태', 0.3342805802822113),
 ('인간', 0.32952404022216797),
 ('변화', 0.32102110981941223),
 ('에너지', 0.31503531336784363),
 ('기능', 0.2980404794216156),
 ('기계', 0.2955460846424103),
 ('연구', 0.2948886752128601),
 ('기술', 0.2920708656311035)]

In [24]:
# 사전학습 된 모델을 사용하여 학습
#사전학습 된 모델 불러오기
pretrained = Word2Vec.load('ko.bin')

print('Pretrained model의 단어 수(V) :', len(pretrained.wv.vocab))
print('Pretrained model의 embedding vector의 size(N) :', len(pretrained.wv['강아지']))

# 모델 생성
model_pretrained = Word2Vec(size = 200, min_count = 0)

# 모델 vocabulary 추가
model_pretrained.build_vocab(corpus_tokenized)

total_examples = model_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_pretrained.wv.vocab))

Pretrained model의 단어 수(V) : 30185
Pretrained model의 embedding vector의 size(N) : 200
현재 vocabulary size(V) : 2725


In [25]:
#pretrained model의 학습되어 있는 vocabulary list 불러오기
model_pretrained.build_vocab([list(pretrained.wv.vocab.keys())], update = True)
print('현재 vocabulary size(V) :', len(model_pretrained.wv.vocab))

#embedding vector 까지 불러오기. lockf = 0.0 이면 불러온 벡터에 대해서는 재학습이 불가함. 1.0이면 재학습 가능.
model_pretrained.intersect_word2vec_format('ko.bin.gz', binary = False, lockf = 1.0)

현재 vocabulary size(V) : 30708


In [26]:
# 모델 학습
model_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 10)

(96060, 96060)

In [32]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_pretrained.wv.most_similar('컴퓨터')

[('하드웨어', 0.7834833264350891),
 ('소프트웨어', 0.7397493124008179),
 ('처리장치', 0.7265453338623047),
 ('마이크로프로세서', 0.7217308282852173),
 ('노트북', 0.7184714078903198),
 ('개인용', 0.7082852721214294),
 ('계산기', 0.7056275606155396),
 ('프린터', 0.6929095983505249),
 ('컴퓨팅', 0.6914974451065063),
 ('전기차', 0.6910395622253418)]

# 3번

사전 학습 없이 모델을 학습해도 성능이 괜찮나요?   
사전 학습 없이 학습한 모델과 사전 학습이 된 모델을 사용한 모델 간의 성능 차이가 있었나요? 차이가 있었다면 어떤 부분에서 차이가 나고 왜 그런 차이가 났는지 생각해보고 설명해주세요!

[여러분의 설명]  
사전 학습된 모델이 더 높은 성능을 내는 것을 확인할 수 있었습니다.  
ex) '컴퓨터'와의 관련성:   
'하드웨어', '소프트웨어' 등의 단어가 '경우', '변화' 등의 단어보다 더 관련이 있을 것 같습니다.   
하지만 사전 학습되지 않은 모델도 꽤 연관성이 높은 단어를 일부 추출해내었습니다.

# 4번

이제 학습된 모델을 통해 나온 embedding 결과를 2차원, 3차원에 embedding 시켜 봅시다.    
사전학습된 데이터를 사용하면 너무 단어 수가 많아지기 때문에 우리가 처음에 가지고 있던 corpus_tokenized 데이터만 사용하겠습니다.   
PCA, t-SNE를 사용해서 시각화하고 그 두가지 방법의 결과를 비교해보세요!   

추가적으로 무언가 재미있는 시각화 결과 (ex. 이거랑 저거랑 비슷한 위치에 임베딩 됐어요!) 가 있다면 추가적으로 남겨주세요!

[여러분의 설명]

In [33]:
# 시각화할 단어 리스트 생성
unique_text = list(set(sum(corpus_tokenized, [])))
len(unique_text)

2725

In [34]:
# embedding 결과 가져오기
word, vector = [], []
for i in unique_text:
    word.append(i)
    vector.append(model_no_pretrained.wv[i])

In [35]:
import plotly
import plotly.graph_objs as go
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [38]:
pca = PCA().fit_transform(vector)
tsne = TSNE(n_components = 2, perplexity = 50, learning_rate = 0.05, init = 'random', n_iter = 500).fit_transform(vector)

In [40]:
# 시각화해보세요!
data = go.Scatter(
    x=pca[:,0],
    y=pca[:,1],
    text = unique_text,
    mode='markers',
    marker=dict(
        size=5,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [41]:
# 시각화해보세요!
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = unique_text,
    mode='markers',
    marker=dict(
        size=5,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

# 5번

t-SNE에는 perplexity라는 hyperparameter가 존재했는데요. 5~50 사이의 값에서 robust 한 결과를 보인다고는 하지만 perplexity가 달라지면서 embedding 결과가 미묘하지만 크게 바뀌기도 합니다. perplexity와 n_iter 조합을 달리해보면서 perplexity의 변화가 embedding결과에 어떤 영향을 미치는지 살펴보세요!

In [46]:
# t-SNE
tsne = TSNE(n_components = 2, perplexity = 5, learning_rate = 0.05, init = 'random', n_iter = 1500).fit_transform(vector)

# 시각화
# 시각화해보세요!
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = unique_text,
    mode='markers',
    marker=dict(
        size=5,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()


/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 수고하셨습니다!